# Example Processing for High-resolution DEM
## Setup and Installation of Python environment
This Notebook and the Python Code to extract Connected Components from longitudinal river profiles requires several python modules to be installed.

Using `conda`, you can create your own environment *DEM_CC* and install the required modules (if you haven't done so already):
```
conda --create DEM_CC
conda activate DEM_CC
conda install -y pandas numpy geopandas rasterio matplotlib scipy kneed sklearn seaborn
```
You can do similar steps in your `pip` environment.

## Installation of LSDTopoTools
Furthermore, the Code requires an installed instance of LSDTopoTools2. We refer to the WEBSITE <LINK>. A few more words about LSDTopoTools and installation
    
## Installing DEM_ConnectedComponents
Download the most recent source code:

```
git clone https://github.com/UP-RS-ESP/DEM_ConnectedComponents.git
```

and install:
```
python setup.py install
```
    
# DEM Preprocessing
There are a few steps that need to be done before the `DEM_connectedcomponents` can be run. These include pre-processing steps with LSDTopoTools or a similar environment.

## `lsdtt-chi-mapping`
Generate a chi map using `lsdtt-chi-mapping`.    
    

In [ ]:
from connectedComponentsFunctions import *

path = "./"
lsdttPath =  "/home/ariane/LSDTopoTools/LSDTopoTools2/LSDTopoTools/LSDTopoTools2/bin/"

#get channel network of testclip. Here I have to set findCompleBasins to true because it is not a closed basin
getLSDTTFlowlines(fname = "testClip", path = path, lsdttPath = lsdttPath, m_over_n= 0.4, findCompleteBasins= "true", testBoundaries= "true")  
lsdttTableTestClip = mergeLSDTToutput(fname = "testClip", path = path, resolution=3, epsg = 32719)    
#find a good pixel threshold for calculating channel slope using 20 random channels within the study area
#taking only a hand-full of channels severely speeds up processing, yet I would reccoment to carefully select a single basin or use the entire study area for more accurate results
pixThr = findPixelThreshold(lsdttTableTestClip, sampleStreams = 20, thresholdRange = np.arange(1,16))

#get channel network of debris-flow sample regions 
getLSDTTFlowlines(fname = "debrisBasins", path = path, lsdttPath = lsdttPath, m_over_n= 0.4)  
lsdttTableDebrisSamples = mergeLSDTToutput(fname = "debrisBasins", path = path, resolution=3, epsg = 32719)   
#use debris-flow sample regions to get a good slope-change threshold to constrain CCs  
dSlopeThr = findDSlopeThreshold(lsdttTableDebrisSamples, pixThr = pixThr, thresholdRange = np.arange(0.15,0.26,0.01))

#run CC Analysis with optimal parameters for both channel networks
runCCAnalysis(fname = "testClip", path = path, lsdttTable = lsdttTableTestClip, pixThr = pixThr, dSlopeThr = dSlopeThr)
runCCAnalysis(fname = "debrisBasins", path = path, lsdttTable = lsdttTableDebrisSamples, pixThr = pixThr, dSlopeThr = dSlopeThr)

#assign debris-flow similarity
dfsiValues = assignDFSI( path = path, allCCName = "testClip", debrisName = "debrisBasins", pixThr = pixThr, dSlopeThr = dSlopeThr)
#backsorting DFSI to stream network
backsorting(fname = "testClip", path = path, dfsiValues = dfsiValues, pixThr = pixThr, dSlopeThr = dSlopeThr)
